### 데이터 불러오기

In [2]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [15]:
df = pd.read_csv('csv/mulit_classification_data.csv')
X_df = df


In [16]:
df

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [17]:
x = X_df.drop(["Pastry","Z_Scratch","K_Scatch","Stains","Dirtiness","Bumps","Other_Faults"], axis=1)
y = df[["Pastry","Z_Scratch","K_Scatch","Stains","Dirtiness","Bumps","Other_Faults"]].copy()

In [18]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
# 표준화 진행 여부
if input_1 == 'y':
    scaler = StandardScaler()
    X = scaler.fit_transform(x)
    print("[안내] 데이터 표준화를 진행했습니다.")
else:
    X = x
    print("[안내] 데이터 표준화를 진행하지 않습니다.")
    
input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")



# 검증 데이터 진행 여부
if input_2 == 'y':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 추가로 분리했습니다.")

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 분리하지 않았습니다.")



[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.


### 모델 구현 
- 임시로 구현

In [27]:
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

# 모델구성
model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim//2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dense(output_dim, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=30, monitor='val_loss')

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

print("validation loss, validation accuracy")
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

           

Epoch 1/1000
39/39 [==============================] - 3s 9ms/step - loss: 1.8856 - accuracy: 0.3368 - val_loss: 1.2985 - val_accuracy: 0.5810
Epoch 2/1000
39/39 [==============================] - 0s 3ms/step - loss: 1.3572 - accuracy: 0.5214 - val_loss: 1.0568 - val_accuracy: 0.6710
Epoch 3/1000
39/39 [==============================] - 0s 3ms/step - loss: 1.1958 - accuracy: 0.5818 - val_loss: 0.9234 - val_accuracy: 0.7044
Epoch 4/1000
39/39 [==============================] - 0s 3ms/step - loss: 1.1176 - accuracy: 0.5826 - val_loss: 0.8614 - val_accuracy: 0.7249
Epoch 5/1000
39/39 [==============================] - 0s 3ms/step - loss: 1.0275 - accuracy: 0.6479 - val_loss: 0.8208 - val_accuracy: 0.7275
Epoch 6/1000
39/39 [==============================] - 0s 3ms/step - loss: 0.9574 - accuracy: 0.6567 - val_loss: 0.7907 - val_accuracy: 0.7404
Epoch 7/1000
39/39 [==============================] - 0s 3ms/step - loss: 0.9380 - accuracy: 0.6785 - val_loss: 0.7943 - val_accuracy: 0.7326
Epoch 